In [4]:
import sys
sys.path.append('../..')
from src import WindSpeedUnit, ForecastAPI, ForecastCurrent, HistoricalAPI, HistoricalDaily, HistoricalHourly
import src
import pandas as pd
import datetime as dt

f = HistoricalAPI()
f.latitude, f.longitude = (53.98352, -6.39139) #src.GEO_COORDINATES.DUNDALK_IT.value
f.wind_speed_unit = WindSpeedUnit.METERS_PER_SECOND
# f.daily = [HistoricalDaily.TEMPERATURE_2M_MEAN, HistoricalDaily.WIND_SPEED_10M_MAX]
f.daily = list(HistoricalDaily._member_map_.values())
# f.hourly = [HistoricalHourly.TEMPERATURE_2M, HistoricalHourly.WIND_SPEED_10M]
f.hourly = list(HistoricalHourly._member_map_.values())
f.start_date = dt.datetime(2020, 1, 1)
f.end_date = dt.datetime(2020, 1, 2)
f.cell_selection = src.enums.CellSelection.NEAREST

response = f.request()
print(f.build_url())
print(response)

https://archive-api.open-meteo.com/v1/archive?cell_selection=nearest&daily=weather_code,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,daylight_duration,sunshine_duration,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration&end_date=2020-01-02&hourly=temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,rain,snowfall,snow_depth,weather_code,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,et0_fao_evapotranspiration,vapour_pressure_deficit,wind_speed_10m,wind_speed_100m,wind_direction_10m,wind_direction_100m,wind_gusts_10m,soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,soil_temperature_28_to_100cm,soil_temperature_100_to_255cm,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,soil_moisture_2

In [5]:
import sqlite3
conn = f._conn

df=pd.read_sql_query("SELECT * FROM hourly_historical_weather_data", conn)
df

,ID,time,latitude,longitude,utc_offset_seconds,timezone,timezone_abbreviation,elevation,parameter,value
0,1,2020-01-01 00:00:00,53.98352,-6.39139,0,GMT,GMT,15.0,albedo,-2.147484e+09
1,2,2020-01-01 01:00:00,53.98352,-6.39139,0,GMT,GMT,15.0,albedo,-2.147484e+09
2,3,2020-01-01 02:00:00,53.98352,-6.39139,0,GMT,GMT,15.0,albedo,-2.147484e+09
3,4,2020-01-01 03:00:00,53.98352,-6.39139,0,GMT,GMT,15.0,albedo,-2.147484e+09
4,5,2020-01-01 04:00:00,53.98352,-6.39139,0,GMT,GMT,15.0,albedo,-2.147484e+09
...,...,...,...,...,...,...,...,...,...,...
1771,1772,2020-01-02 19:00:00,53.98352,-6.39139,0,GMT,GMT,15.0,wind_speed_10m,2.060000e+01
1772,1773,2020-01-02 20:00:00,53.98352,-6.39139,0,GMT,GMT,15.0,wind_speed_10m,2.060000e+01
1773,1774,2020-01-02 21:00:00,53.98352,-6.39139,0,GMT,GMT,15.0,wind_speed_10m,1.980000e+01
1774,1775,2020-01-02 22:00:00,53.98352,-6.39139,0,GMT,GMT,15.0,wind_speed_10m,1.950000e+01


In [4]:
df.drop(columns = 'ID').loc[lambda self: self.duplicated(keep = 'first')].sort_values(by = 'time')
print(pd.to_numeric.__doc__)


    Convert argument to a numeric type.

    The default return dtype is `float64` or `int64`
    depending on the data supplied. Use the `downcast` parameter
    to obtain other dtypes.

    Please note that precision loss may occur if really large numbers
    are passed in. Due to the internal limitations of `ndarray`, if
    numbers smaller than `-9223372036854775808` (np.iinfo(np.int64).min)
    or larger than `18446744073709551615` (np.iinfo(np.uint64).max) are
    passed in, it is very likely they will be converted to float so that
    they can be stored in an `ndarray`. These warnings apply similarly to
    `Series` since it internally leverages `ndarray`.

    Parameters
    ----------
    arg : scalar, list, tuple, 1-d array, or Series
        Argument to be converted.
    errors : {'ignore', 'raise', 'coerce'}, default 'raise'
        - If 'raise', then invalid parsing will raise an exception.
        - If 'coerce', then invalid parsing will be set as NaN.
        - If 'igno

In [ ]:
df.duplicated().sum()
conn.close()

: 

In [10]:
hist = HIST
# hist.time.values[-1]
f._conn.close()
hist

,time,latitude,longitude,utc_offset_seconds,timezone,timezone_abbreviation,elevation,parameter,value
0,2000-01-01 00:00:00,53.983520,-6.391390,0,GMT,GMT,15.0,temperature_2m,4.8
1,2000-01-01 01:00:00,53.983520,-6.391390,0,GMT,GMT,15.0,temperature_2m,3.9
2,2000-01-01 02:00:00,53.983520,-6.391390,0,GMT,GMT,15.0,temperature_2m,3.4
3,2000-01-01 03:00:00,53.983520,-6.391390,0,GMT,GMT,15.0,temperature_2m,2.8
4,2000-01-01 04:00:00,53.983520,-6.391390,0,GMT,GMT,15.0,temperature_2m,2.2
...,...,...,...,...,...,...,...,...,...
1551050,2020-01-02T19:00,53.954304,-6.441009,0,GMT,GMT,15.0,wind_gusts_10m,41.8
1551051,2020-01-02T20:00,53.954304,-6.441009,0,GMT,GMT,15.0,wind_gusts_10m,40.0
1551052,2020-01-02T21:00,53.954304,-6.441009,0,GMT,GMT,15.0,wind_gusts_10m,40.0
1551053,2020-01-02T22:00,53.954304,-6.441009,0,GMT,GMT,15.0,wind_gusts_10m,38.2


In [4]:
'https://archive-api.open-meteo.com/v1/archive?cell_selection=nearest&daily=temperature_2m_mean,wind_speed_10m_max&end_date=2020-01-02&hourly=temperature_2m,wind_speed_10m&latitude=50.98352&longitude=6.39139&latitude=53.98352&longitude=-6.39139&start_date=2020-01-01&wind_speed_unit=ms'

'https://archive-api.open-meteo.com/v1/archive?cell_selection=nearest&daily=temperature_2m_mean,wind_speed_10m_max&end_date=2020-01-02&hourly=temperature_2m,wind_speed_10m&latitude=50.98352&longitude=6.39139&latitude=53.98352&longitude=-6.39139&start_date=2020-01-01&wind_speed_unit=ms'

In [5]:
url = 'https://archive-api.open-meteo.com/v1/archive?latitude=52.0&longitude=12.0&latitude=53.98352&longitude=-6.39139&start_date=2025-02-01&end_date=2025-02-19&hourly=temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,rain,snowfall,snow_depth,weather_code,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,et0_fao_evapotranspiration,vapour_pressure_deficit,wind_speed_10m,wind_speed_100m,wind_direction_10m,wind_direction_100m,wind_gusts_10m,soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,soil_temperature_28_to_100cm,soil_temperature_100_to_255cm,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,soil_moisture_28_to_100cm,soil_moisture_100_to_255cm,boundary_layer_height,wet_bulb_temperature_2m,total_column_integrated_water_vapour,is_day,sunshine_duration,albedo,snow_depth_water_equivalent&daily=weather_code,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,daylight_duration,sunshine_duration,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration&wind_speed_unit=ms&timezone=auto'
src.weather.utils.ApiCounter.calculate_call_weight_from_url(url) #3718.842 is the answer. It's wrong though.
# src.GEO_COORDINATES.DUNDALK_IT.value

# pd.read_pickle('historical.pkl').longitude[0]


14.657142857142858

In [14]:
import datetime as dt

url = f.build_url()

parsedParams = src.weather.utils.ApiCounter.parse_url_params(url)
startDate = dt.datetime.strptime(parsedParams['start_date'], '%Y-%m-%d')#.strftime('%Y-%m-%d %H:%M:%S')
endDate = dt.datetime.strptime(parsedParams['end_date'], '%Y-%m-%d')#.strftime('%Y-%m-%d %H:%M:%S')
parsedParams['start_date'] = startDate.strftime('%Y-%m-%d %H:%M:%S')
parsedParams['end_date'] = endDate.strftime('%Y-%m-%d %H:%M:%S')

queryDaily = pd.DataFrame(data = pd.date_range(start=startDate, end=endDate, freq='D'), columns=['time'])
queryDaily = queryDaily.merge(pd.DataFrame(data = parsedParams['daily'],columns = ['parameter']), how='cross')
queryDaily = queryDaily.merge(
    pd.DataFrame(dict(latitude = parsedParams['latitude'], longitude = parsedParams['longitude'])),
    how='cross')
queryDaily.time = queryDaily.time.dt.strftime('%Y-%m-%d %H:%M:%S')


queryHourly = pd.DataFrame(data = pd.date_range(start=startDate,end=endDate, freq='h'), columns=['time'])
queryHourly = queryHourly.merge(pd.DataFrame(data = parsedParams['hourly'],columns = ['parameter']), how='cross')
queryHourly = queryHourly.merge(
    pd.DataFrame(dict(latitude = parsedParams['latitude'], longitude = parsedParams['longitude'])),
    how='cross')
queryHourly.time = queryHourly.time.dt.strftime('%Y-%m-%d %H:%M:%S')

#ensure dtypes
queryDaily['time'] = queryDaily['time'].astype('str')
queryDaily['parameter'] = queryDaily['parameter'].astype('str')
queryDaily['latitude'] = queryDaily['latitude'].astype('float')
queryDaily['longitude'] = queryDaily['longitude'].astype('float')

queryHourly['time'] = queryHourly['time'].astype('str')
queryHourly['parameter'] = queryHourly['parameter'].astype('str')
queryHourly['latitude'] = queryHourly['latitude'].astype('float')
queryHourly['longitude'] = queryHourly['longitude'].astype('float')

# make stging tables
conn = f._conn
cursor = conn.cursor()

hourlyStageCommand = '''
CREATE TABLE IF NOT EXISTS hourly_historical_weather_staging_table(
    time TEXT,
    parameter TEXT,
    latitude REAL,
    longitude REAL
);
'''
cursor.execute(hourlyStageCommand)
conn.commit()

dailyStageCommand = '''
CREATE TABLE IF NOT EXISTS daily_historical_weather_staging_table(
    time TEXT,
    parameter TEXT,
    latitude REAL,
    longitude REAL
);
'''
cursor.execute( dailyStageCommand)
conn.commit()

#insert data
queryDaily.to_sql('daily_historical_weather_staging_table', conn, if_exists='replace', index=False)
queryHourly.to_sql('hourly_historical_weather_staging_table', conn, if_exists='replace', index=False)

925

In [15]:
# create a command that joins the staging table (request table) with the historical data and then pull it out
EPSILON = 1E-12
# dailySearchCommand = '''
# SELECT * FROM (
#         SELECT * FROM daily_historical_weather_data
#         WHERE time BETWEEN ? AND ?
#     ) AS DAILY
# LEFT JOIN daily_historical_weather_staging_table
# ON daily_historical_weather_staging_table.time = DAILY.time
# AND daily_historical_weather_staging_table.parameter = DAILY.parameter
# AND daily_historical_weather_staging_table.latitude = DAILY.latitude
# AND daily_historical_weather_staging_table.longitude = DAILY.longitude
# '''

dailySearchCommand = f'''
SELECT
    COALESCE(DAILY.time, daily_historical_weather_staging_table.time) AS time,
    COALESCE(DAILY.parameter, daily_historical_weather_staging_table.parameter) AS parameter,
    COALESCE(DAILY.latitude, daily_historical_weather_staging_table.latitude) AS latitude,
    COALESCE(DAILY.longitude, daily_historical_weather_staging_table.longitude) AS longitude,
    DAILY.value
FROM (
    SELECT time, parameter, latitude, longitude, value
    FROM daily_historical_weather_data
    WHERE time BETWEEN ? AND ?
) AS DAILY
RIGHT JOIN daily_historical_weather_staging_table
ON daily_historical_weather_staging_table.time = DAILY.time
AND daily_historical_weather_staging_table.parameter = DAILY.parameter
AND ABS(daily_historical_weather_staging_table.latitude - DAILY.latitude) < {EPSILON}
AND ABS(daily_historical_weather_staging_table.longitude - DAILY.longitude) < {EPSILON}
'''
responseDaily = pd.read_sql_query(dailySearchCommand, conn, params=[parsedParams['start_date'], parsedParams['end_date']])


hourlySearchCommand = f'''
SELECT 
    COALESCE(HOURLY.time, hourly_historical_weather_staging_table.time) AS time,
    COALESCE(HOURLY.parameter, hourly_historical_weather_staging_table.parameter) AS parameter,
    COALESCE(HOURLY.latitude, hourly_historical_weather_staging_table.latitude) AS latitude,
    COALESCE(HOURLY.longitude, hourly_historical_weather_staging_table.longitude) AS longitude,
    HOURLY.value
FROM (
    SELECT time, parameter, latitude, longitude, value
    FROM hourly_historical_weather_data
    WHERE time BETWEEN ? AND ?
) AS HOURLY
RIGHT JOIN hourly_historical_weather_staging_table
ON hourly_historical_weather_staging_table.time = HOURLY.time
AND hourly_historical_weather_staging_table.parameter = HOURLY.parameter
AND ABS(hourly_historical_weather_staging_table.latitude - HOURLY.latitude) < {EPSILON}
AND ABS(hourly_historical_weather_staging_table.longitude - HOURLY.longitude) < {EPSILON}
'''

responseHourly = pd.read_sql_query(hourlySearchCommand, conn, params=[parsedParams['start_date'], parsedParams['end_date']])

In [16]:
df

,ID,time,latitude,longitude,utc_offset_seconds,timezone,timezone_abbreviation,elevation,parameter,value
0,1,2020-01-01 00:00:00,53.98352,-6.39139,0,GMT,GMT,15.0,albedo,NaN
1,2,2020-01-01 01:00:00,53.98352,-6.39139,0,GMT,GMT,15.0,albedo,NaN
2,3,2020-01-01 02:00:00,53.98352,-6.39139,0,GMT,GMT,15.0,albedo,NaN
3,4,2020-01-01 03:00:00,53.98352,-6.39139,0,GMT,GMT,15.0,albedo,NaN
4,5,2020-01-01 04:00:00,53.98352,-6.39139,0,GMT,GMT,15.0,albedo,NaN
...,...,...,...,...,...,...,...,...,...,...
1963,1964,2020-01-02 19:00:00,53.98352,-6.39139,0,GMT,GMT,15.0,snow_depth_water_equivalent,NaN
1964,1965,2020-01-02 20:00:00,53.98352,-6.39139,0,GMT,GMT,15.0,snow_depth_water_equivalent,NaN
1965,1966,2020-01-02 21:00:00,53.98352,-6.39139,0,GMT,GMT,15.0,snow_depth_water_equivalent,NaN
1966,1967,2020-01-02 22:00:00,53.98352,-6.39139,0,GMT,GMT,15.0,snow_depth_water_equivalent,NaN


In [17]:
responseHourly#.groupby('time').agg(parameter = ('parameter', 'count'))

,time,parameter,latitude,longitude,value
0,2020-01-01 00:00:00,albedo,53.98352,-6.39139,NaN
1,2020-01-01 01:00:00,albedo,53.98352,-6.39139,NaN
2,2020-01-01 02:00:00,albedo,53.98352,-6.39139,NaN
3,2020-01-01 03:00:00,albedo,53.98352,-6.39139,NaN
4,2020-01-01 04:00:00,albedo,53.98352,-6.39139,NaN
...,...,...,...,...,...
1020,2020-01-01 20:00:00,snow_depth_water_equivalent,53.98352,-6.39139,NaN
1021,2020-01-01 21:00:00,snow_depth_water_equivalent,53.98352,-6.39139,NaN
1022,2020-01-01 22:00:00,snow_depth_water_equivalent,53.98352,-6.39139,NaN
1023,2020-01-01 23:00:00,snow_depth_water_equivalent,53.98352,-6.39139,NaN


In [18]:
# foundHourly.loc[lambda s:s.isna().any(axis=1)]
foundHourly = responseHourly.copy().loc[lambda s:s.value.notna()]
foundDaily = responseDaily.copy().loc[lambda s:s.value.notna()]
notFoundHourly = responseHourly.copy().loc[lambda s:s.value.isna()]
notFoundDaily = responseDaily.copy().loc[lambda s:s.value.isna()]

# now what? make a serparate requests for the missing data. No, too computationally expensive to split up all the data.
# Furthermore, I will get punished by the api for splitting the requests up too finely.
# this means I'd have to spend ages figuring out how to optimally split the requests up to minimize api calling.
# I will just look at at it like this: 
    # I am expecting to make historical ts requests with long ranges, freqently.
    # I am assuming that 10% of the data is missing. And what is present is continuous. So 'ill just request from min notFoundDaily.time.min() onwards.
    # after submitting to database I will tell db to drop duplicates.



In [20]:
foundHourly

,time,parameter,latitude,longitude,value
25,2020-01-01 00:00:00,apparent_temperature,53.98352,-6.39139,2.5
26,2020-01-01 01:00:00,apparent_temperature,53.98352,-6.39139,2.9
27,2020-01-01 02:00:00,apparent_temperature,53.98352,-6.39139,3.5
28,2020-01-01 03:00:00,apparent_temperature,53.98352,-6.39139,3.8
29,2020-01-01 04:00:00,apparent_temperature,53.98352,-6.39139,3.7
...,...,...,...,...,...
920,2020-01-01 20:00:00,wind_speed_10m,53.98352,-6.39139,11.1
921,2020-01-01 21:00:00,wind_speed_10m,53.98352,-6.39139,12.0
922,2020-01-01 22:00:00,wind_speed_10m,53.98352,-6.39139,13.0
923,2020-01-01 23:00:00,wind_speed_10m,53.98352,-6.39139,13.7


In [23]:
notFoundHourly.groupby('time').agg(parameter = ('parameter', lambda s: ','.join(set(s))))

,parameter
time,
2020-01-01 00:00:00,"albedo,snow_depth_water_equivalent"
2020-01-01 01:00:00,"albedo,snow_depth_water_equivalent"
2020-01-01 02:00:00,"albedo,snow_depth_water_equivalent"
2020-01-01 03:00:00,"albedo,snow_depth_water_equivalent"
2020-01-01 04:00:00,"albedo,snow_depth_water_equivalent"
2020-01-01 05:00:00,"albedo,snow_depth_water_equivalent"
2020-01-01 06:00:00,"albedo,snow_depth_water_equivalent"
2020-01-01 07:00:00,"albedo,snow_depth_water_equivalent"
2020-01-01 08:00:00,"albedo,snow_depth_water_equivalent"


In [20]:
pd.read_sql_query('SELECT * FROM daily_historical_weather_data', conn,index_col='ID') 

,time,latitude,longitude,utc_offset_seconds,timezone,timezone_abbreviation,elevation,parameter,value
ID,,,,,,,,,
1,2020-01-01 00:00:00,53.954304,-6.441009,0,GMT,GMT,15.0,apparent_temperature_max,3.9
2,2020-01-02 00:00:00,53.954304,-6.441009,0,GMT,GMT,15.0,apparent_temperature_max,7.6
3,2020-01-01 00:00:00,53.954304,-6.441009,0,GMT,GMT,15.0,apparent_temperature_mean,2.5
4,2020-01-02 00:00:00,53.954304,-6.441009,0,GMT,GMT,15.0,apparent_temperature_mean,5.9
5,2020-01-01 00:00:00,53.954304,-6.441009,0,GMT,GMT,15.0,apparent_temperature_min,-0.1
...,...,...,...,...,...,...,...,...,...
236,2020-01-02 00:00:00,53.954304,-6.391390,0,GMT,GMT,15.0,wind_direction_10m_dominant,208.0
237,2020-01-01 00:00:00,53.954304,-6.391390,0,GMT,GMT,15.0,wind_gusts_10m_max,25.2
238,2020-01-02 00:00:00,53.954304,-6.391390,0,GMT,GMT,15.0,wind_gusts_10m_max,51.1


In [21]:
pd.read_sql_query('SELECT * FROM hourly_historical_weather_data', conn)

,time,latitude,longitude,utc_offset_seconds,timezone,timezone_abbreviation,elevation,parameter,value
0,2000-01-01 00:00:00,53.983520,-6.39139,0,GMT,GMT,15.0,temperature_2m,4.8
1,2000-01-01 01:00:00,53.983520,-6.39139,0,GMT,GMT,15.0,temperature_2m,3.9
2,2000-01-01 02:00:00,53.983520,-6.39139,0,GMT,GMT,15.0,temperature_2m,3.4
3,2000-01-01 03:00:00,53.983520,-6.39139,0,GMT,GMT,15.0,temperature_2m,2.8
4,2000-01-01 04:00:00,53.983520,-6.39139,0,GMT,GMT,15.0,temperature_2m,2.2
...,...,...,...,...,...,...,...,...,...
1551050,2020-01-02 19:00:00,53.954304,-6.39139,0,GMT,GMT,15.0,wind_gusts_10m,41.8
1551051,2020-01-02 20:00:00,53.954304,-6.39139,0,GMT,GMT,15.0,wind_gusts_10m,40.0
1551052,2020-01-02 21:00:00,53.954304,-6.39139,0,GMT,GMT,15.0,wind_gusts_10m,40.0
1551053,2020-01-02 22:00:00,53.954304,-6.39139,0,GMT,GMT,15.0,wind_gusts_10m,38.2


In [9]:
notFoundDaily

,time,parameter,latitude,longitude,value
0,2020-01-01 00:00:00,weather_code,53.98352,-6.39139,None
1,2020-01-01 00:00:00,temperature_2m_max,53.98352,-6.39139,None
2,2020-01-01 00:00:00,temperature_2m_min,53.98352,-6.39139,None
3,2020-01-01 00:00:00,temperature_2m_mean,53.98352,-6.39139,None
4,2020-01-01 00:00:00,apparent_temperature_max,53.98352,-6.39139,None
5,2020-01-01 00:00:00,apparent_temperature_min,53.98352,-6.39139,None
6,2020-01-01 00:00:00,apparent_temperature_mean,53.98352,-6.39139,None
7,2020-01-01 00:00:00,sunrise,53.98352,-6.39139,None
8,2020-01-01 00:00:00,sunset,53.98352,-6.39139,None
9,2020-01-01 00:00:00,daylight_duration,53.98352,-6.39139,None


In [180]:
# notFoundHourly = notFoundHourly.assign(geo = lambda s: s.latitude.astype('str') + ', ' + s.longitude.astype('str'))
# notFoundHourly.sort_values(by = 'geo parameter time'.split(), inplace=True, ignore_index=True)
# notFoundHourly['timeObj'] = pd.to_datetime(notFoundHourly.time)


# for key,grp in notFoundHourly.groupby('geo parameter'.split()):
#     break
# # (grp.timeObj.shift(-1)-grp.timeObj).fillna(pd.Timedelta(hours=1))
# grp['dt'] = (grp.timeObj.shift(-1)-grp.timeObj).fillna(pd.Timedelta(hours=1))
# grp['dt'] = grp.dt.dt.total_seconds()/3600
# grp

In [207]:
df = notFoundHourly.copy()
df = df.assign(geo = lambda s: s.latitude.astype('str') + ',' + s.longitude.astype('str'))
df.sort_values(by = 'geo parameter time'.split(), inplace=True, ignore_index=True)
df['timeObj'] = pd.to_datetime(df.time)
df = df.groupby('geo timeObj'.split()).agg(parameter = ('parameter',','.join)).reset_index()
df = df.groupby('geo parameter'.split()).agg(start_date = ('timeObj','min'), end_date = ('timeObj','max')).reset_index()
df = df.groupby('parameter start_date end_date'.split()).agg(geo = ('geo',' '.join)).reset_index()

row = df.iloc[0]
for geo in row['geo'].split(' '):
    latitude, longitude = geo.split(',')
    hourly = [HistoricalHourly._value2member_map_[value] for value in row['parameter'].split(',')]
    sd = row['start_date'].to_pydatetime()
    ed = row['end_date'].to_pydatetime()

In [43]:
df = foundHourly.copy()
df = df.assign(geo = lambda s: s.latitude.astype('str') + ',' + s.longitude.astype('str'))
gdf = df.copy()
gdf = gdf.assign(geo = lambda s: s.latitude.astype('str') + ',' + s.longitude.astype('str'))
res = (
    gdf.groupby('geo'.split())
        .agg(
                latitude = ('latitude','first'), 
                longitude = ('longitude','first'),
            )
        .assign(
                generationtime_ms = -10.0,
                utc_offset_seconds= -10,
                timezone= 'GMT',
                timezone_abbreviation= 'GMT',
                elevation= -9999,
                daily_units= lambda df:df.shape[0]*[{'time': 'iso8601','temperature_2m_mean': '°C','wind_speed_10m_max': 'km/h'}],            
        )
        .reset_index(drop=True)
        .to_dict(orient='records')
)
df.time = pd.to_datetime(df.time).dt.strftime('%Y-%m-%dT%H:%M:%S')
for dict_,(_,grp) in zip(res,df.groupby('geo')):
    dict_['daily'] = grp.pivot_table(index = 'time', columns = 'parameter', values = 'value', aggfunc='first').reset_index().to_dict(orient='list')

In [3]:
df

NameError: name 'df' is not defined